# 适配器模式

## 目标

将一个类型的接口转换到另外一个类型的接口，提供给调用者使用。额外提供的适配器不需要服务提供者和调用者修改任何代码。

In [1]:
// 水形态枚举类
public enum WaterState {
    SOLID, LIQUID, GAS;
    @Override
    public String toString() {
        switch(this) {
            case SOLID:
                return "固态";
            case LIQUID:
                return "液态";
            case GAS:
                return "气态";
            default:
                return null;
        }
    }
}

In [2]:
// 甲的API提供了使用开尔文温度表示的水的形态接口
public interface JudgeWaterStateKelvin {
    WaterState getStateByKelvin(double kelvin);
}

// 甲的实现类
public class KelvinWaterJudger implements JudgeWaterStateKelvin {
    @Override
    public WaterState getStateByKelvin(double kelvin) {
        if(kelvin < 273.15) {
            return WaterState.SOLID;
        } else if(kelvin < 373.15) {
            return WaterState.LIQUID;
        } else {
            return WaterState.GAS;
        }
    }
}

In [3]:
// 乙的API提供了使用摄氏度表示的水的形态接口，调用方原本调用的就是这个接口
public interface JudgeWaterStateCelsius {
    WaterState getStateByCelsius(double kelvin);
}

// 乙的实现类
public class CelsiusWaterJudger implements JudgeWaterStateCelsius {
    @Override
    public WaterState getStateByCelsius(double celsius) {
        if(celsius < 0) {
            return WaterState.SOLID;
        } else if(celsius < 100) {
            return WaterState.LIQUID;
        } else {
            return WaterState.GAS;
        }
    }
}

In [4]:
// 为了让甲和调用方都不修改代码，添加一个适配器类
public class KelvinWaterAdapter implements JudgeWaterStateCelsius {
    private KelvinWaterJudger kelvinJudger = new KelvinWaterJudger();
    @Override
    public WaterState getStateByCelsius(double celsius) {
        double kelvin = celsius + 273.15;
        return kelvinJudger.getStateByKelvin(kelvin);
    }
}

In [5]:
// 调用者类
public class WaterStateClient {
    private JudgeWaterStateCelsius judger;
    public WaterStateClient(JudgeWaterStateCelsius judger) {
        this.judger = judger;
    }
    public void setJudger(JudgeWaterStateCelsius judger) {
        this.judger = judger;
    }
    public WaterState getState(double temp) {
        return judger.getStateByCelsius(temp);
    }
}

In [6]:
// 测试主类
public class App {
    public static void main(String[] args) {
        // 原来调用乙API的方式
        WaterStateClient client = new WaterStateClient(new CelsiusWaterJudger());
        test(client);
        // 现在调用甲API的方式，甲API代码完全不用修改，调用方代码基本不用修改
        client.setJudger(new KelvinWaterAdapter());
        test(client);
    }
    
    public static void test(WaterStateClient client) {
        System.out.println(String.format("%.2f 摄氏度 水形态为 %s", -10.0, client.getState(-10)));
        System.out.println(String.format("%.2f 摄氏度 水形态为 %s", 37.5, client.getState(37.5)));
        System.out.println(String.format("%.2f 摄氏度 水形态为 %s", 234.1, client.getState(234.1)));
    }
}

In [7]:
new App().main(null)

-10.00 摄氏度 水形态为 固态
37.50 摄氏度 水形态为 液态
234.10 摄氏度 水形态为 气态
-10.00 摄氏度 水形态为 固态
37.50 摄氏度 水形态为 液态
234.10 摄氏度 水形态为 气态
